In [13]:
## installing required libraries
!pip install cassandra-driver pandas

Defaulting to user installation because normal site-packages is not writeable


In [3]:
## Connect to AstraDB
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd

# Path to your secure connect bundle
ASTRA_DB_BUNDLE_PATH = 'secure-connect-hello.zip'

# Set up the authentication and cluster
def create_session():
    cloud_config = {
        'secure_connect_bundle': ASTRA_DB_BUNDLE_PATH
    }
    auth_provider = PlainTextAuthProvider('makUuXvvkAWdDyKrsMOvharq', '9W6d+7JwkA.HGaJOd62ZXjlvUx9xt_4W5_ctJtw4ES6.CFxK36huoyps6N9GzhtMlkEvRnpD4Nz_BnRc2NJK1Z+D47KsICzqrcZQ23WqZr+mHtZb2I,WqNDYF3,DvJK,')
    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    session = cluster.connect()
    return session

session = create_session()


In [4]:
## Read data from CSV File
import uuid

# Load your CSV file
df = pd.read_csv(r'C:\Users\DELL\sales_100.csv')

# View first few rows to understand the structure
df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,UnitsSold,UnitPrice,UnitCost,TotalRevenue,TotalCost,TotalProfit
0,Sub-Saharan Africa,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.33,6.92,14862.69,11023.56,3839.13
1,Middle East and North Africa,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
2,Australia and Oceania,Papua New Guinea,Meat,Offline,M,5/15/2015,940995585,6/4/2015,360,421.89,364.69,151880.40,131288.40,20592.00
3,Sub-Saharan Africa,Djibouti,Clothes,Offline,H,5/17/2017,880811536,7/2/2017,562,109.28,35.84,61415.36,20142.08,41273.28
4,Europe,Slovakia,Beverages,Offline,L,10/26/2016,174590194,12/4/2016,3973,47.45,31.79,188518.85,126301.67,62217.18


In [5]:
## Set Keyspace
session.set_keyspace('hello')


In [6]:
## Inserting data from csv file to create bronze level 
from cassandra.query import SimpleStatement
import uuid

insert_query = session.prepare("""
    INSERT INTO sales_databronze(id, region, country, item_type, sales_channel, order_priority, order_date, order_id,
                       ship_date, units_sold, unit_price, unit_cost, total_revenue, total_cost, total_profit)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

# Insert each row from DataFrame
for _, row in df.iterrows():
    session.execute(insert_query, (
        uuid.uuid4(),
        row['Region'],
        row['Country'],
        row['Item Type'],
        row['Sales Channel'],
        row['Order Priority'],
        str(row['Order Date']),
        int(row['Order ID']),
        str(row['Ship Date']),
        int(row['UnitsSold']),
        float(row['UnitPrice']),
        float(row['UnitCost']),
        float(row['TotalRevenue']),
        float(row['TotalCost']),
        float(row['TotalProfit']),
    ))

In [7]:
## verifying that the data is inserted into Bronze Level table 
rows = session.execute("SELECT * FROM sales_databronze LIMIT 5")
for row in rows:
    print(row)

Row(id=UUID('8c2c2b8f-aaca-4265-9254-ec2b3c162a2a'), country='Iceland', item_type='Baby Food', order_date='10/2/2010', order_id=678230941, order_priority='M', region='Europe', sales_channel='Offline', ship_date='11/3/2010', total_cost=392492.04, total_profit=236007.32, total_revenue=628499.36, unit_cost=159.42, unit_price=255.28, units_sold=2462)
Row(id=UUID('0b77c299-eaff-4d17-938f-3069295eaa79'), country='Italy', item_type='Cereal', order_date='11/30/2015', order_id=887409770, order_priority='C', region='Europe', sales_channel='Online', ship_date='12/5/2015', total_cost=366085.86, total_profit=276932.34, total_revenue=643018.2, unit_cost=117.11, unit_price=205.7, units_sold=3126)
Row(id=UUID('1ae23538-6990-4f76-b7c9-4ebc3ee2bbdc'), country='Greece', item_type='Cereal', order_date='8/22/2015', order_id=887124383, order_priority='H', region='Europe', sales_channel='Online', ship_date='10/8/2015', total_cost=1015812.14, total_profit=768429.66, total_revenue=1784241.8, unit_cost=117.11, 

In [14]:
## Inserting data into silver table 
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Load cleaned Silver layer data
df = pd.read_csv(r'C:\Users\DELL\sales_silver.csv')

# Path to your secure connect bundle (unzipped folder)
SECURE_CONNECT_BUNDLE_PATH = r'C:\Users\DELL\secure-connect-hello.zip'

# Set up Cassandra cluster with authentication
cloud_config = {
    'secure_connect_bundle': SECURE_CONNECT_BUNDLE_PATH
}

auth_provider = PlainTextAuthProvider('makUuXvvkAWdDyKrsMOvharq', '9W6d+7JwkA.HGaJOd62ZXjlvUx9xt_4W5_ctJtw4ES6.CFxK36huoyps6N9GzhtMlkEvRnpD4Nz_BnRc2NJK1Z+D47KsICzqrcZQ23WqZr+mHtZb2I,WqNDYF3,DvJK,')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Set keyspace
session.set_keyspace('hello')

# Prepare insert statement
insert_stmt = session.prepare("""
    INSERT INTO sales_datasilver (
        order_id, region, country, item_type, sales_channel, order_priority,
        order_date, ship_date, units_sold, unit_price, unit_cost,
        total_revenue, total_cost, total_profit
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

# Insert rows into Astra
for _, row in df.iterrows():
    session.execute(insert_stmt, (
        int(row['order_id']),
        row['region'],
        row['country'],
        row['item_type'],
        row['sales_channel'],
        row['order_priority'],
        row['order_date'],   # should be in 'YYYY-MM-DD'
        row['ship_date'],
        int(row['units_sold']),
        float(row['unit_price']),
        float(row['unit_cost']),
        float(row['total_revenue']),
        float(row['total_cost']),
        float(row['total_profit'])
    ))

print("✅ Data loaded successfully to Astra DB!")


✅ Data loaded successfully to Astra DB!


In [18]:
rows = session.execute("SELECT * FROM sales_datasilver")
data = [dict(row._asdict()) for row in rows]
df = pd.DataFrame(data)

In [19]:
# Clean Dates
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")
df["ship_date"] = pd.to_datetime(df["ship_date"], errors="coerce")

# Drop nulls in critical columns
df.dropna(subset=["order_date", "ship_date", "units_sold", "unit_price", "unit_cost"], inplace=True)

# Drop duplicates
df.drop_duplicates(inplace=True)

# Fix order_priority
priority_map = {"L": "Low", "M": "Medium", "H": "High", "C": "Critical"}
df["order_priority"] = df["order_priority"].map(priority_map).fillna("Unknown")

# Normalize text
df["sales_channel"] = df["sales_channel"].str.strip().str.lower()
df["item_type"] = df["item_type"].str.strip().str.title()

# Calculate delivery_days
df["delivery_days"] = (df["ship_date"] - df["order_date"]).dt.days

# Remove negative values
df = df[
    (df["total_revenue"] >= 0) &
    (df["total_cost"] >= 0) &
    (df["total_profit"] >= 0)
]

In [22]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd

# (Assuming your session is already created)

# ⚠️ This removes all rows
session.execute("TRUNCATE sales_silver")

print("✅ Table truncated. Ready to insert cleaned data.")

✅ Table truncated. Ready to insert cleaned data.


In [25]:
session.execute("ALTER TABLE sales_datasilver ADD delivery_days int;")

In [32]:
# Load original file
df = pd.read_csv("sales_silver.csv")

# Cleaning steps
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")
df["ship_date"] = pd.to_datetime(df["ship_date"], errors="coerce")
df.dropna(subset=["order_date", "ship_date", "units_sold", "unit_price", "unit_cost"], inplace=True)
df.drop_duplicates(inplace=True)
df["order_priority"] = df["order_priority"].map({"L": "Low", "M": "Medium", "H": "High", "C": "Critical"}).fillna("Unknown")
df["sales_channel"] = df["sales_channel"].str.strip().str.lower()
df["item_type"] = df["item_type"].str.strip().str.title()
df["delivery_days"] = (df["ship_date"] - df["order_date"]).dt.days
df = df[
    (df["total_revenue"] >= 0) &
    (df["total_cost"] >= 0) &
    (df["total_profit"] >= 0)
]

# Save to cleaned file
df.to_csv("sales_silver_cleaned.csv", index=False)

In [34]:
import pandas as pd

# Load raw (bronze) data
df_raw = pd.read_csv("sales_silver.csv")
print(f"🔹 Raw rows: {len(df_raw)}")

# Make a copy to clean
df = df_raw.copy()

# 1. Convert order_date and ship_date to datetime
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")
df["ship_date"] = pd.to_datetime(df["ship_date"], errors="coerce")

before = len(df)
df = df.dropna(subset=["order_date", "ship_date"])
print(f"🧹 Removed rows with invalid dates: {before - len(df)}")

# 2. Drop rows with nulls in important numeric columns
important_cols = ["units_sold", "unit_price", "unit_cost"]
before = len(df)
df = df.dropna(subset=important_cols)
print(f"🧹 Removed rows with missing values in {important_cols}: {before - len(df)}")

# 3. Drop duplicates
before = len(df)
df = df.drop_duplicates()
print(f"🧹 Removed duplicate rows: {before - len(df)}")

# 4. Standardize order_priority
priority_map = {"L": "Low", "M": "Medium", "H": "High", "C": "Critical"}
df["order_priority"] = df["order_priority"].map(priority_map).fillna("Unknown")

# 5. Add delivery_days column
df["delivery_days"] = (df["ship_date"] - df["order_date"]).dt.days

# 6. Drop rows with negative values in financial columns
financial_cols = ["total_revenue", "total_cost", "total_profit"]
before = len(df)
df = df[
    (df["total_revenue"] >= 0) &
    (df["total_cost"] >= 0) &
    (df["total_profit"] >= 0)
]
print(f"🧹 Removed rows with negative values in {financial_cols}: {before - len(df)}")

# 7. Normalize text fields
df["sales_channel"] = df["sales_channel"].str.strip().str.lower()
df["item_type"] = df["item_type"].str.strip().str.title()

# Save cleaned data
df.to_csv("sales_silver_cleaned.csv", index=False)
print(f"\n✅ Cleaned data saved to 'sales_silver_cleaned.csv'")
print(f"📊 Final cleaned row count: {len(df)}")


🔹 Raw rows: 99
🧹 Removed rows with invalid dates: 0
🧹 Removed rows with missing values in ['units_sold', 'unit_price', 'unit_cost']: 0
🧹 Removed duplicate rows: 0
🧹 Removed rows with negative values in ['total_revenue', 'total_cost', 'total_profit']: 0

✅ Cleaned data saved to 'sales_silver_cleaned.csv'
📊 Final cleaned row count: 99


In [35]:
import pandas as pd

df = pd.read_csv("sales_silver_cleaned.csv")
print(f"✅ Rows in cleaned CSV: {len(df)}")

✅ Rows in cleaned CSV: 99


In [36]:
print(df_raw.isnull().sum())

region            0
country           0
item_type         0
sales_channel     0
order_priority    0
order_date        0
order_id          0
ship_date         0
units_sold        0
unit_price        0
unit_cost         0
total_revenue     0
total_cost        0
total_profit      0
dtype: int64


In [37]:
print(df_raw.duplicated().sum())

0


In [38]:
print((df_raw[["total_revenue", "total_cost", "total_profit"]] < 0).sum())

total_revenue    0
total_cost       0
total_profit     0
dtype: int64


In [39]:
### Gold table

import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# 🔹 Step 1: Load Cleaned Data
df = pd.read_csv("sales_silver_cleaned.csv")

# 🔹 Step 2: Aggregate - Total profit by region
region_profit = df.groupby("region")["total_profit"].sum().reset_index()

# 🟡 Optional: Round values
region_profit["total_profit"] = region_profit["total_profit"].round(2)

# 🔹 Step 3: Connect to Datastax Astra DB
ASTRA_DB_SECURE_BUNDLE_PATH = r'C:/Users/DELL/secure-connect-hello.zip'  # Update to your actual path
ASTRA_DB_CLIENT_ID = 'makUuXvvkAWdDyKrsMOvharq'  # Paste from your DB credentials
ASTRA_DB_CLIENT_SECRET = '9W6d+7JwkA.HGaJOd62ZXjlvUx9xt_4W5_ctJtw4ES6.CFxK36huoyps6N9GzhtMlkEvRnpD4Nz_BnRc2NJK1Z+D47KsICzqrcZQ23WqZr+mHtZb2I,WqNDYF3,DvJK,'

cloud_config = {
    'secure_connect_bundle': ASTRA_DB_SECURE_BUNDLE_PATH
}

auth_provider = PlainTextAuthProvider(ASTRA_DB_CLIENT_ID, ASTRA_DB_CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('hello')  # Replace with your keyspace name

# 🔹 Step 4: Insert aggregated data into Gold table
insert_query = "INSERT INTO sales_gold_region_profit (region, total_profit) VALUES (?, ?)"
prepared = session.prepare(insert_query)

for _, row in region_profit.iterrows():
    session.execute(prepared, (row["region"], float(row["total_profit"])))

print("✅ Gold data inserted into 'sales_gold_region_profit'")


✅ Gold data inserted into 'sales_gold_region_profit'


In [40]:
### Gold level Average units sold by item type
avg_units = df.groupby("item_type")["units_sold"].mean().reset_index()
avg_units["avg_units_sold"] = avg_units["units_sold"].round(2)

insert_query = "INSERT INTO sales_gold_avg_units (item_type, avg_units_sold) VALUES (?, ?)"
prepared = session.prepare(insert_query)

for _, row in avg_units.iterrows():
    session.execute(prepared, (row["item_type"], float(row["avg_units_sold"])))

In [41]:
### Gold level third table Monthly revenue
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# 🔹 Load cleaned silver data
df = pd.read_csv("sales_silver_cleaned.csv")

# 🔹 Create year_month column (format: YYYY-MM)
df["order_date"] = pd.to_datetime(df["order_date"])
df["year_month"] = df["order_date"].dt.to_period("M").astype(str)

# 🔹 Group by year_month and calculate total revenue
monthly_revenue = df.groupby("year_month")["total_revenue"].sum().reset_index()
monthly_revenue["total_revenue"] = monthly_revenue["total_revenue"].round(2)

# 🔹 Connect to Astra DB
ASTRA_DB_SECURE_BUNDLE_PATH = 'C:/Users/DELL/secure-connect-hello.zip'  # Update path
ASTRA_DB_CLIENT_ID = 'makUuXvvkAWdDyKrsMOvharq'         # Replace with your client id
ASTRA_DB_CLIENT_SECRET = '9W6d+7JwkA.HGaJOd62ZXjlvUx9xt_4W5_ctJtw4ES6.CFxK36huoyps6N9GzhtMlkEvRnpD4Nz_BnRc2NJK1Z+D47KsICzqrcZQ23WqZr+mHtZb2I,WqNDYF3,DvJK,' # Replace with your secret

cloud_config = {
    'secure_connect_bundle': ASTRA_DB_SECURE_BUNDLE_PATH
}
auth_provider = PlainTextAuthProvider(ASTRA_DB_CLIENT_ID, ASTRA_DB_CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('hello')  # replace with your keyspace

# 🔹 Insert into Gold table
insert_query = "INSERT INTO sales_gold_monthly_revenue (year_month, total_revenue) VALUES (?, ?)"
prepared = session.prepare(insert_query)

for _, row in monthly_revenue.iterrows():
    session.execute(prepared, (row["year_month"], float(row["total_revenue"])))

print("✅ Monthly revenue inserted into 'sales_gold_monthly_revenue'")


✅ Monthly revenue inserted into 'sales_gold_monthly_revenue'
